In [1]:
from transformers import T5ForConditionalGeneration, T5Tokenizer, Trainer, TrainingArguments
import torch
from datasets import DatasetDict, Dataset
#new tokenizer 
from transformers import AutoTokenizer 
model = T5ForConditionalGeneration.from_pretrained("t5-small")
tokenizer = AutoTokenizer.from_pretrained("t5-small",additional_special_tokens=None,extra_ids=0,model_max_length=1e30,)
print(tokenizer.additional_special_tokens)
['<extra_id_0>',
 '<extra_id_1>',
 '<extra_id_2>',
 '<extra_id_3>',]
#old tokenizer used 
# model = T5ForConditionalGeneration.from_pretrained("t5-small")
# tokenizer = T5Tokenizer.from_pretrained("t5-small")
# Define your mental health dataset
MHdataset = [
    {"source": "What are some common signs of anxiety?", "target": "Common signs of anxiety include restlessness, excessive worry, and increased heart rate."},
    {"source": "What are some common signs of anxiety?", "target": "Common signs of anxiety include restlessness, excessive worry, and increased heart rate."},
    {"source": "How can I manage stress?", "target": "Managing stress involves practicing relaxation techniques, exercising, and seeking support from friends or professionals."},
    {"source": "What are the symptoms of depression?", "target": "Symptoms of depression may include persistent sadness, loss of interest in activities, and changes in appetite."},
    {"source": "How can I improve my mental health?", "target": "You can improve your mental health by maintaining a balanced lifestyle, seeking help when needed, and practicing self-care."},
    {"source": "What is the importance of self-care?", "target": "Self-care is essential for maintaining mental and emotional well-being. It helps reduce stress and improve overall quality of life."},
    {"source": "Are there any natural remedies for anxiety?", "target": "Yes, some natural remedies for anxiety include deep breathing exercises, mindfulness meditation, and herbal teas like chamomile or lavender."},
    {"source": "What is the difference between stress and anxiety?", "target": "Stress is a response to a specific event or situation, while anxiety is a prolonged feeling of unease and worry. Both can have physical and emotional symptoms."},
    {"source": "How can I deal with social anxiety?", "target": "Dealing with social anxiety involves gradual exposure to social situations, cognitive-behavioral therapy, and learning relaxation techniques."},
    {"source": "What role does exercise play in mental health?", "target": "Regular exercise can boost mood, reduce anxiety and depression, and improve overall mental health."},
    {"source": "Is it common to experience mood swings?", "target": "Yes, occasional mood swings are common, but persistent and extreme mood swings may be a sign of a mood disorder that requires professional help."},
    {"source": "How can I help a friend who is struggling with mental health issues?", "target": "Offer support, listen without judgment, encourage them to seek professional help, and check in regularly."},
    {"source": "What is the impact of a healthy diet on mental health?", "target": "A healthy diet can improve mental health by providing essential nutrients for brain function and stabilizing mood."},
    {"source": "Can music or art therapy help with mental health?", "target": "Yes, music and art therapy can be beneficial in expressing emotions and reducing stress and anxiety."},
    {"source": "What is the relationship between sleep and mental health?", "target": "Sleep plays a crucial role in mental health. Poor sleep can worsen existing mental health issues, while good sleep promotes emotional well-being."},
    {"source": "Are there effective techniques for coping with grief?", "target": "Coping with grief involves allowing oneself to grieve, seeking support, and finding healthy ways to remember and honor the loved one."},
    {"source": "How can I build resilience against life's challenges?", "target": "Building resilience involves developing problem-solving skills, seeking social support, and maintaining a positive outlook."},
    {"source": "What are the benefits of therapy or counseling?", "target": "Therapy can provide a safe space to discuss and address mental health concerns, improve coping skills, and promote self-awareness."},
    {"source": "Is it normal to have occasional moments of self-doubt?", "target": "Yes, self-doubt is a common experience. It's important to challenge negative self-talk and seek support when needed."},
    {"source": "What are the effects of prolonged stress on the body?", "target": "Prolonged stress can lead to physical and mental health issues, such as high blood pressure, insomnia, and increased risk of anxiety and depression."},
    {"source": "How can I practice mindfulness in daily life?", "target": "Practicing mindfulness involves being present in the moment, focusing on your breath, and observing your thoughts without judgment."},
]




[]


In [2]:
# Convert your dataset to a pandas DataFrame
import pandas as pd
df = pd.DataFrame(MHdataset)

# Create a Dataset object from your pandas DataFrame
train_dataset = Dataset.from_pandas(df)

# Convert your Dataset object to a DatasetDict object
dataset = DatasetDict({"train": train_dataset})

# Select the train split
train_split = dataset["train"]

# Access the first item in the train split
first_item = train_split[0]

train_split
print(len(train_dataset))



21


In [3]:
# def my_data_collator(batch):
#     source_ids = []
#     for source in batch["source"]:
#         source_ids.append(tokenizer(source)["input_ids"])

#     batch["source"] = source_ids

#     # Move this code inside the function
#     inputs = tokenizer(batch["source"], return_tensors="pt", padding=True, max_length=200)
#     return {"input_ids": inputs["input_ids"]}
#      # inputs = tokenizer(batch["source"], return_tensors="pt", padding=True, max_length=200)
#     # return {"input_ids": inputs["input_ids"]}




In [4]:
#finetuning the model
def my_data_collator(batch):
    source_texts = [example["source"] for example in batch]
    target_texts = [example["target"] for example in batch]

    inputs = tokenizer(source_texts, return_tensors="pt", padding=True, truncation=True, max_length=200)
    labels = tokenizer(target_texts, return_tensors="pt", padding=True, truncation=True, max_length=200)

    return {
        "input_ids": inputs["input_ids"],
        "attention_mask": inputs["attention_mask"],
        "labels": labels["input_ids"],
    }


training_args = TrainingArguments(
    output_dir="./chatbot_model",
    overwrite_output_dir=True,
    num_train_epochs=400,
    per_device_train_batch_size= 20,
    save_steps=10,
    save_total_limit=2,
    remove_unused_columns=False,
    
)

print(getattr(training_args, "per_device_train_batch_size"))


trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=my_data_collator,
    train_dataset=train_dataset,
    tokenizer=tokenizer,
)


# if len(train_dataset) > 0:
#     trainer.train()
# else:
#     print("The train_dataset dataset is empty. Please add some data to the dataset before training the model.")


trainer.train()


20


Step,Training Loss
500,1.633900


TrainOutput(global_step=800, training_loss=1.1841931533813477, metrics={'train_runtime': 453.953, 'train_samples_per_second': 18.504, 'train_steps_per_second': 1.762, 'total_flos': 36994835546112.0, 'train_loss': 1.1841931533813477, 'epoch': 400.0})

In [ ]:
# Greet the user
print("Chatbot: Hey, how can I assist you today?")
while True:
    user_input = input("You: ")

    # Define the input text
    input_text =  user_input

    # Encode the input text
    input_ids = tokenizer.encode(input_text, return_tensors="pt", max_length=512, truncation=True)

    # Generate a response using the fine-tuned model
    response_ids = model.generate(input_ids, max_length=50, num_beams=4, no_repeat_ngram_size=2, top_k=50, top_p=0.9, do_sample=True)

    chatbot_response = tokenizer.decode(response_ids[0], skip_special_tokens=True)
    print("Chatbot:", chatbot_response)

    
    
#Old
# while True:
#     user_input = input("You: ")
#     input_text = "Q: " + user_input
#     input_ids = tokenizer.encode(input_text, return_tensors="pt", max_length=512, truncation=True)
#     response_ids = model.generate(input_ids, max_length=50, num_beams=4, no_repeat_ngram_size=2, top_k=50, top_p=0.95, eos_token_id=tokenizer.eos_token_id)

#     chatbot_response = tokenizer.decode(response_ids[0], skip_special_tokens=True)
#     print("Chatbot:", chatbot_response)


Chatbot: Hey, how can I assist you today?


You:  What are the symptoms of depression?


Chatbot: Symptoms of depression may include persistent persistent sadness, loss of appetite, and changes in appetite.


You:  What are the symptoms of depression?


Chatbot: Symptoms of depression may include persistent sadness, loss of interest in activities, and changes in appetite.


...

Chatbot: Stress is a response to the specific needs of people, while anxiety is intense. It can also cause anxiety to increase in frequency and intensity. Both can have physical and emotional symptoms.


You:  Is it common to experience mood swings?


Chatbot: Est-ce common to experience mood swings?


You:  Is it common to experience mood swings?


Chatbot: Yes, experiencing mood swings is a common experience.


In [ ]:
# print("Chatbot: Hey, how can I assist you today?")


# MHdataset = [
#     "Q: What are some common signs of anxiety?\nA: Common signs of anxiety include restlessness, excessive worry, and increased heart rate.",
#     "Q: How can I manage stress?\nA: Managing stress involves practicing relaxation techniques, exercising, and seeking support from friends or professionals.",
#     "Q: What are the symptoms of depression?\nA: Symptoms of depression may include persistent sadness, loss of interest in activities, and changes in appetite.",
#     "Q: How can I improve my mental health?\nA: You can improve your mental health by maintaining a balanced lifestyle, seeking help when needed, and practicing self-care.",
# ]

# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=mh.dataset,
# )
# trainer.train()


# while True:
#     user_input = input("You: ")
#     input_text = "chatbot: " + user_input
#     input_ids = tokenizer.encode(input_text, return_tensors="pt", max_length=512, truncation=True)
#     response_ids = model.generate(input_ids, max_length=50, num_beams=4, no_repeat_ngram_size=2, top_k=50, top_p=0.95, eos_token_id=tokenizer.eos_token_id)

#     chatbot_response = tokenizer.decode(response_ids[0], skip_special_tokens=True)
#     print("Chatbot:", chatbot_response)
    
# response_ids = model.generate(
#     input_ids, 
#     max_length=50, 
#     num_beams=4, 
#     do_sample=True,  # Set do_sample to True for sampling-based generation
#     top_p=0.95,       # Adjust top_p as needed
#     no_repeat_ngram_size=2, 
#     top_k=50, 
#     eos_token_id=tokenizer.eos_token_id
# )
